# Кластеризація. Лабораторна робота №3.

## 1. Імпортуємо необхідні бібліотеки


In [168]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

## 2. Читаємо датасет з інформацією про людей, що отримана з Вікіпедії.

In [148]:
# Читаємо датасет
people = pd.read_csv("./people_wiki.csv")
people

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former australian rules footballer who played with the kangaroos and carlton in the australian football league aflfrom western australia morrell played his early senior football for west perth his 44game senior career for the falcons spanned 19982000 and he was the clubs leading goalkicker in 2000 at the age of 21 morrell was recruited to the australian football league by the kangaroos football club with its third round selection in the 2001 afl rookie draft as a forward he twice kicked five goals during his time with the kangaroos the first was in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell was traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 games for the blues before being delisted at the end of 2005 he continued to play victorian football league vfl football with the northern bullants carltons vflaffiliate in 2006 and acted as playing assistant coach in 2007 in 2008 he shifted to the box hill hawks before retiring from playing at the end of the season from 2009 until 2013 morrell was the senior coach of the strathmore football club in the essendon district football league leading the club to the 2011 premier division premiership since 2014 he has coached the west coburg football club also in the edflhe currently teaches physical education at parade college in melbourne
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from university of chicago in 1973 after studying psychiatry pharmacology and ophthalmology he is a full professor and vicechair of the department of psychiatry at ohsu oregon health science university and holds an md and phd prior to moving to oregon in 1981 lewy was at the national institute of mental health nimh in bethesda maryland working with senior colleague thomas wehr in oregon he has worked closely with robert l sack as of december 2005 he had 94 publications available on pubmed he describes his research as follows my laboratory studies chronobiologic sleep and mood disorders these disorders include winter depression jet lag maladaptation to shift work and certain types of sleep disturbances relying on a very precise assay for plasma melatonin a hormone that has a clearly defined 24hour pattern of secretion biological rhythm disorders can be assessed and their treatment can be monitored current research is focused on developing bright light exposure and melatonin administration as treatment modalities for these disorders treatment must be precisely scheduled morning light exposure and evening melatonin administration cause circadian phaseadvance shifts evening light exposure and morning melatonin administration cause circadian phasedelay shifts totally blind individuals have 25hour circadian rhythms drifting an hour later each day unless they take a melatonin capsule at a certain time every day
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player who has been active in canadas blues scene since 1982 hailing from vancouver he crossed tens of thousands of miles playing club dates and festivals in canada the northwestern united states and germanyover the years he has issued seven cds in 1995 his home is where the harp is won the muddy award for the best nw blues release from the cascade blues association in portland oregon as well that year it was nominated for a canadian juno for the best bluesgospel recording teamed up with graham guest on piano his cd naturally was voted 1 canadian blues album of 2010 by the blind lemon surveybrown tours extensively with his guitarist j arthur edmonds performing their electric mid1950s chicago blues either as a duo or with the full band while he is home he juggles a few combos working many venues big and sm

In [149]:
# Знаходимо статтю про конкретну людину Барак Обама
obama = people[people.name == "Barack Obama"].copy()
obama.iloc[0]

URI                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## 3. Отримаємо кількість слів для статті Обами


In [150]:
# Векторизація документа за допомогою техніки BAG_OF_WORDS
vectorizer = CountVectorizer()
# Вивчення всіх унікальних слів в тексті (створення словника) та перетворення тексту в числовий вектор, де кожен елемент кількість появ відповідного слова =>
# отримуємо розріджену матрицю частот (зберігає тільки ненульові значення та їх індекси)
features = vectorizer.fit_transform(obama.text)
vocab = vectorizer.get_feature_names_out()
# Загальна кількість разів коли кожне слово зустрілося
dist = np.sum(features.toarray(), axis=0)

In [151]:
# Створюємо таблицю частот слів
obama_word_count_table = pd.DataFrame({'token': vocab, 'count': dist})
# Відсортуємо її за зменшенням кількості слів та виведемо 10 найчастіших слів
obama_word_count_table.sort_values('count', ascending=False).head(10)

,token,count
242,the,40
115,in,30
28,and,21
162,of,18
245,to,14
106,his,11
160,obama,9
18,act,8
104,he,7
133,law,6


## 4. Розрахуємо TF_IDF для всієї таблиці (всього корпусу документів)


In [152]:
vectorizer_tf = TfidfVectorizer(stop_words="english") # Автоматично обчислює TF, зменшує вагу слів якщо низький IDF, одразу виключаємо стоп-слова, які не несуть жодної цінності для алгоритму
dtm = vectorizer_tf.fit_transform(people.text) # TF_IDF матриця (розріджена)

In [153]:
dtm.shape # Розмірність матриці

(59071, 548115)

In [154]:
# Отримуємо список всіх слів, які були знайдені у документах
features_tf = vectorizer_tf.get_feature_names_out()

## 5. Вивчимо TF_IDF для статті Обами

In [155]:
obama

,URI,name,text
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born august 4 1961 is the 44th and current president of the united states and the first african american to hold the office born in honolulu hawaii obama is a graduate of columbia university and harvard law school where he served as president of the harvard law review he was a community organizer in chicago before earning his law degree he worked as a civil rights attorney and taught constitutional law at the university of chicago law school from 1992 to 2004 he served three terms representing the 13th district in the illinois senate from 1997 to 2004 running unsuccessfully for the united states house of representatives in 2000in 2004 obama received national attention during his campaign to represent illinois in the united states senate with his victory in the march democratic party primary his keynote address at the democratic national convention in july and his election to the senate in november he began his presidential campaign in 2007 and after a close primary campaign against hillary rodham clinton in 2008 he won sufficient delegates in the democratic party primaries to receive the presidential nomination he then defeated republican nominee john mccain in the general election and was inaugurated as president on january 20 2009 nine months after his election obama was named the 2009 nobel peace prize laureateduring his first two years in office obama signed into law economic stimulus legislation in response to the great recession in the form of the american recovery and reinvestment act of 2009 and the tax relief unemployment insurance reauthorization and job creation act of 2010 other major domestic initiatives in his first term included the patient protection and affordable care act often referred to as obamacare the doddfrank wall street reform and consumer protection act and the dont ask dont tell repeal act of 2010 in foreign policy obama ended us military involvement in the iraq war increased us troop levels in afghanistan signed the new start arms control treaty with russia ordered us military involvement in libya and ordered the military operation that resulted in the death of osama bin laden in january 2011 the republicans regained control of the house of representatives as the democratic party lost a total of 63 seats and after a lengthy debate over federal spending and whether or not to raise the nations debt limit obama signed the budget control act of 2011 and the american taxpayer relief act of 2012obama was reelected president in november 2012 defeating republican nominee mitt romney and was sworn in for a second term on january 20 2013 during his second term obama has promoted domestic policies related to gun control in response to the sandy hook elementary school shooting and has called for full equality for lgbt americans while his administration has filed briefs which urged the supreme court to strike down the defense of marriage act of 1996 and californias proposition 8 as unconstitutional in foreign policy obama ordered us military involvement in iraq in response to gains made by the islamic state in iraq after the 2011 withdrawal from iraq continued the process of ending us combat operations in afghanistan and has sought to normalize us relations with cuba


In [156]:
# Знаходимо індекс документа про Обаму
obama_index = obama.index[0]
# Створюємо словник для збереження tf_idf значень
word_scores = {}
for i, word in enumerate(features_tf):
    word_scores[word] = dtm[obama_index, i]
# Створюємо таблицю для відображення tf_idf значень
obama_word_scores = pd.DataFrame(word_scores.items(), columns=['word', 'tfidf'])
obama_word_scores.sort_values('tfidf', ascending=False).head(10)

,word,tfidf
358378,obama,0.413495
45070,act,0.282170
259085,iraq,0.171970
292368,law,0.163903
138523,control,0.149369
365249,ordered,0.138633
328423,military,0.135368
155872,democratic,0.129792
417604,response,0.124821
258530,involvement,0.124821


## 6. Розрахунок міри подібності між людьми (Обамою, Клінтоном та Бекхемом) на базі TF_IDF за допомогою функції `cosine_similarity`.

In [157]:
clinton = people[people.name == "Bill Clinton"]
beckham = people[people.name == "David Beckham"]

# Косинусна міра подібності вимірює схожість між двома векторами,
# обчислюючи косинус кута між ними
sim_1 = cosine_similarity(dtm[obama.index], dtm[clinton.index])
sim_2 = cosine_similarity(dtm[obama.index], dtm[beckham.index])

print(f"Косинусна подібність між Обамою та Клінтоном: {sim_1[0]}")
print(f"Косинусна подібність між Обамою та Бекхемом: {sim_2[0]}")

Косинусна подібність між Обамою та Клінтоном: [0.18896718]
Косинусна подібність між Обамою та Бекхемом: [0.02556581]


## 7. Використання моделі NNM (Nearest Neighbour Model)

In [158]:
# Створюємо модель пошуку сусідів, орієнтовану на пошук 10 найближчих
knn_model = NearestNeighbors(n_neighbors=10)
# Навчаємо модель на TF_IDF матриці - надаємо простір для пошуку
knn_model.fit(dtm)

NearestNeighbors(n_neighbors=10)

## 8. Шукаємо найближчих людей до Обами

In [159]:
# Шукаємо найближчих до Обами
distances, indices = knn_model.kneighbors(dtm[obama.index], return_distance=True)
# Виводимо таблицю знайдених найближчих людей
nearest_df = pd.DataFrame({'names': people.loc[indices[0]].name, "distance": distances[0]})
nearest_df.sort_values(by="distance").head(10)

,names,distance
35817,Barack Obama,0.000000
24478,Joe Biden,1.165145
38376,Samantha Power,1.207369
57108,Hillary Rodham Clinton,1.219640
38714,Eric Stern (politician),1.222509
46140,Robert Gibbs,1.236178
18827,Henry Waxman,1.243057
44681,Jesse Lee (politician),1.244667
6796,Eric Holder,1.248296
2412,Joe the Plumber,1.251607


## Домашнє завдання

 1. Вивести перших 5 слів і їх кількість в тексті про людину по варіанту (__#5 Carrie Underwood__) при представленні тексту за допомогою dtm, а також і TF_IDF.
 Вивести текст про задану людину.

In [160]:
# Виводимо текст про задану людину
carrie = people[people.name == "Carrie Underwood"]
carrie.iloc[0].text

'carrie marie underwood born march 10 1983 is an american singer songwriter and actress she rose to fame as the winner of the fourth season of american idol in 2005 underwood has since become one of the most prominent mainstream recording artists in the music industry setting and breaking several records throughout her career her achievements led her to be inducted into the grand ole opry in 2008 and into the oklahoma music hall of fame in 2009 she has won a vast array of awards including six grammy awards sixteen billboard music awards eleven academy of country music awards and eight american music awards along with being nominated for a golden globe award for best original songher debut album some hearts was released in 2005 bolstered by the huge crossover success of the singles jesus take the wheel and before he cheats it went on to become the fastest selling debut country album in nielsen soundscan history the bestselling solo female debut album in country music history and the bes

In [161]:
# Представляємо текст за допомогою dtm (document-term matrix)
vectorizer_count = CountVectorizer()
# Створюємо словник для статті про Керрі та векторизуємо документ (за BAG_OF_WORDS)
features_1 = vectorizer_count.fit_transform(carrie.text)
vocab_1 = vectorizer_count.get_feature_names_out()
# Загальна кількість разів коли кожне слово зустрілося
count = np.sum(features_1.toarray(), axis=0)
# Створюємо табличне представлення dtm
carrie_word_count_table = pd.DataFrame({'token': vocab_1, 'count': count})
carrie_word_count_table.sort_values("count", ascending=False).head(5)

,token,count
184,the,36
24,and,16
130,of,15
98,in,12
56,country,11


In [162]:
# Аналізуємо TF_IDF для даної статті про Керрі
# Створюємо словник для збереження tf_idf значень
word_scores_1 = {}
for i, word in enumerate(features_tf):
    word_scores_1[word] = dtm[carrie.index[0], i]
# Створюємо таблицю для відображення tf_idf значень
carrie_word_count_table = pd.DataFrame(word_scores_1.items(), columns=["token", "tf_idf"])
carrie_word_count_table.sort_values("tf_idf", ascending=False).head(5)

,token,tf_idf
508279,underwood,0.519588
141830,country,0.314403
194489,female,0.210140
89613,billboard,0.207110
52523,album,0.187929


2. Вивести відстань між двома людьми по варіанту (__#5 Carrie Underwood and Kelly Ayotte__), також текст про другу людину і провести інтерпретацію результату.

In [163]:
# Знаходимо рядок Kelly Ayotte та виводимо текст
kelly = people[people.name == "Kelly Ayotte"]
kelly.iloc[0].text

'kelly ann ayotte et ayot born june 27 1968 is an american politician and the junior united states senator from new hampshire serving since 2011 a member of the republican party she is the second youngest of the 20 female senators and the eleventhyoungest overallborn in nashua new hampshire ayotte is a graduate of pennsylvania state university and villanova university school of law she worked as a law clerk for the new hampshire supreme court before entering private practice she also worked as a prosecutor for the new hampshire department of justice and briefly served as the legal counsel to new hampshire governor craig benson before returning to the department of justice to serve as deputy attorney general of new hampshire in june 2004 governor benson appointed ayotte as attorney general of new hampshire after the resignation of peter heed she became new hampshires first woman attorney general serving from 2004 to 2009 after she was twice reappointed by democratic governor john lynch 

In [164]:
# Знаходимо подібність (відстань) між Керрі та Келлі за косинусною мірою подібності
sim_result = cosine_similarity(dtm[carrie.index], dtm[kelly.index])
print(f"Косинусна подібність між Керрі та Келлі: {sim_result[0,0]:.3f}")

Косинусна подібність між Керрі та Келлі: 0.016


#### Інтерпретація результатів
Отримане значення косинусної подібності 0.016 вказує на дуже низьку схожість між текстами про Керрі та Келлі. Це означає, що ці тексти майже не мають спільних слів або тем.

3. Побудувати NNM-модель на базі TF_IDF (використовуючи міру для відстані
cosine). Знайти найближчих сусідів для людини по варіанту із таблиці завдання Вивести тексти для перших двох найближчих сусідів, перевірити правильність роботи моделі.

In [165]:
# Будуємо NNM-модель
knn_model_1 = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_model_1.fit(dtm)
# Шукаємо найближчих людей до Келлі
distances_1, indices_1 = knn_model_1.kneighbors(dtm[carrie.index], return_distance=True)
# Виводимо таблицю знайдених найближчих людей
carrie_nearest_df = pd.DataFrame({'names': people.loc[indices_1[0]].name, "distance": distances_1[0]})
carrie_nearest_df.sort_values(by="distance").head(5)

,names,distance
317,Carrie Underwood,0.000000
29297,Kelly Clarkson,0.632841
508,Faith Hill,0.652212
27793,Adele,0.653539
38592,Billy Ray Cyrus,0.653639


In [166]:
# Виводимо текст для перших двох найближчих сусідів
kelly_clarkson = people[people.name == "Kelly Clarkson"]
kelly_clarkson.iloc[0].text

'kelly brianne clarkson born april 24 1982 is an american singer and songwriter in 2002 she rose to fame after winning the first season of american idol and has since been established as the original american idol her debut single a moment like this topped the us billboard hot 100 and broke the record for the biggest jump to number one in the charts history it became the bestselling single of the year in the country she became the runnerup of world idol the following yearclarksons debut studio album thankful 2003 has been certified 2 platinum and sold over 45 million copies internationally its lead single miss independent became an international hit earning clarkson her first grammy nomination she developed a rockoriented sound with the release of her second album breakaway 2004 it has been certified 6 platinum and sold over 15 million copies worldwide earning clarkson two grammy awards including one for the hit single since u been gone she later took full creative direction of her thi

In [167]:
faith_hill = people[people.name == "Faith Hill"]
faith_hill.iloc[0].text

'faith hill born audrey faith perry september 21 1967 is an american country pop singer and occasional actress she is one of the most successful country artists of all time having sold more than 40 million records worldwide hill is married to country singer tim mcgraw with whom she has recorded several successful duetshills first two albums take me as i am 1993 and it matters to me 1995 established her as a popular country singer they placed a combined three number ones on billboards country charts and were major successes in north america she later rose to mainstream crossover and international fame with the release of her next two albums faith 1998 and breathe 1999 faith spawned her first international hit this kiss and went multiplatinum in various countries breathe became her bestselling album to date and one of the bestselling country albums of all time with the huge crossover success of her signature song breathe it had massive sales worldwide and earned hill three grammy awards 

#### Інтерпретація результатів
Найближчими сусідами є: Kelly Clarkson — відстань ≈ 0.63, Faith Hill — відстань ≈ 0.65.

Це означає, що статті про Kelly Clarkson та Faith Hill найбільш схожі за вмістом на текст про Carrie Underwood з усієї вибірки.

Після виведення змісту текстів, можна зрозуміти, що тексти мають семантичну схожість та переконатися в правильності роботи моделі.